In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy

In [2]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [3]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [4]:
df_city.head(20)

,City,lat,lng
0,fredericksburg,30.530832,-99.061504
1,lumberton,30.167681,-94.288827
2,uvalde,29.109367,-99.652922
3,robinson,31.204904,-96.792679
4,seguin,29.334885,-97.594407
5,liberty,30.239979,-94.676481
6,port arthur,29.412456,-94.114376
7,lufkin,31.238026,-94.406888
8,copperas cove,31.287755,-97.951227
9,sulphur,30.139013,-93.377064


In [5]:
#check length of cities
len(list(df_city["City"].unique()))

113

In [6]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['fredericksburg',
 'lumberton',
 'uvalde',
 'robinson',
 'seguin',
 'liberty',
 'port+arthur',
 'lufkin',
 'copperas+cove',
 'sulphur',
 'tomball',
 'jacksonville',
 'lockhart',
 'leander',
 'victoria',
 'bryan',
 'leon+valley',
 'huntsville',
 'nederland',
 'san+angelo',
 'belton',
 'kerrville',
 'dickinson',
 'beaumont',
 'hondo',
 'orange',
 'canyon+lake',
 'beeville',
 'galveston',
 'conroe',
 'gatesville',
 'brownwood',
 'brenham',
 'el+campo',
 'temple',
 'college+station',
 'nacogdoches',
 'stephenville',
 'austin',
 'palestine',
 'san+antonio',
 'pflugerville',
 'bay+city',
 'wharton',
 'la+marque',
 'angleton',
 'lake+jackson',
 'atascocita',
 'natchitoches',
 'pleasanton',
 'corsicana',
 'missouri+city',
 'killeen',
 'freeport',
 'mission+bend',
 'del+rio',
 'bridge+city',
 'taylor',
 'humble',
 'friendswood',
 'alvin',
 'san+marcos',
 'bellmead',
 'woodway',
 'katy',
 'vidor',
 'kirby',
 'sugar+land',
 'bellaire',
 'lakeway',
 'rosenberg',
 'baytown',
 'port+lavaca',
 'sant

In [7]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [12]:
 base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
temp=[]
wind_speed=[]
pressure=[]
city_name=[]

for city in target_cities[:4]:
    query_url=f"{base_current_url}appid={weather_api_key}&q={city},US&units=imperial"
    req = requests.get(query_url)
    response1=req.json()
    latitude=response1["coord"]["lat"]
    longitude=response1["coord"]["lon"]

    for x in unix_time():
        historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
        hist_json = requests.get(historical_url).json()
        temp.append(hist_json["current"]["temp"])
        pressure.append(hist_json["current"]["pressure"])
        wind_speed.append(hist_json["current"]["wind_speed"])
        city_name.append(response1['name'])
        print(historical_url)

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=38.3032&lon=-77.4605&dt=1630299600&appid=15a347531c589488d7fec7ef38ad8fac&units=imperial
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=38.3032&lon=-77.4605&dt=1630213200&appid=15a347531c589488d7fec7ef38ad8fac&units=imperial
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=38.3032&lon=-77.4605&dt=1630126800&appid=15a347531c589488d7fec7ef38ad8fac&units=imperial
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=38.3032&lon=-77.4605&dt=1630040400&appid=15a347531c589488d7fec7ef38ad8fac&units=imperial
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=38.3032&lon=-77.4605&dt=1629954000&appid=15a347531c589488d7fec7ef38ad8fac&units=imperial
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=34.6182&lon=-79.0086&dt=1630299600&appid=15a347531c589488d7fec7ef38ad8fac&units=imperial
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=34.6182&lon=-79.0086&dt=1

In [17]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    city_name=[]
    country_code=[]
    for city in target_cities[:5]:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},US-TX,US&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
            
            
        except:
            print("City not found. Skipping...")
            pass
        for x in unix_time():
            historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
            hist_json = requests.get(historical_url).json()
            temp.append(hist_json["current"]["temp"])
            pressure.append(hist_json["current"]["pressure"])
            wind_speed.append(hist_json["current"]["wind_speed"])
            city_name.append(response1['name'])
            country_code.append(response1["sys"]["country"])
    d=dict()
    d["City Name"] = city_name
    d["Temperature"] = temp
    d["Wind_Speed"] = wind_speed
    d["Pressure"] = pressure    
    return(d)
        #print ("--"*30)

In [18]:
df =pd.DataFrame(find_location(target_cities))

KeyError: 'current'

In [ ]:
df